In [8]:
import pyodbc
import distance
import jieba
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import synonyms
from gensim.models import *
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pandas import DataFrame
import pandas as pd

cnxn = pyodbc.connect(r'Driver=SQL Server;Server=LAPTOP-VFEIS9GU;Database=DIET;Trusted_Connection=yes;') #連線DB
cursor = cnxn.cursor()
cursor.execute("SELECT Name FROM MOS")

corpus_data = []
result = cursor.fetchall()  #讀DB
for row in result:
    corpus_data.append(row)

#字串處理
corpus_data=str(corpus_data).replace('(','').replace(')','').replace(',, ',',').replace('[','').replace(']','').replace("'","").replace(' ','')  
corpus_data=corpus_data.split(',')
corpus_data=list(corpus_data)
k=1
for i in range(len(corpus_data)):
    if i%2==1:
        del corpus_data[k]
        k=k+1

path= '7_Algorithm.txt'                   #寫入w2v需要的txt作為語料庫

with open('7_Algorithm.txt','a+',encoding='UTF-8') as test: #清空
    test.truncate(0)
    
for write in corpus_data:                                    #寫入
    f = open(path, 'a',encoding='UTF-8')
    f.write(write)
    f.write('\n')
f.close()
train_data = word2vec.LineSentence('7_Algorithm.txt')

cnxn.close()

In [12]:
def Levenshtein(keyword):
    def edit_distance(corpus_data, keyword):
        return distance.levenshtein(corpus_data, keyword)
    i=0
    for i in range(10):
        i=i+1
        results = list(filter(lambda x: edit_distance(x, keyword) <= i, corpus_data))
        if(len(results)>9):
#             print("step",i,results)
            break
        else:
#             print("step",i,"未達top10")
            continue

    long=len(results)           #create等長list
    Similarity=['-']*long           

    #df sort
    dict_df1={
             "Similarity":Similarity,   
             "Name":results
            }
    data1=DataFrame(dict_df1)
    sult1=data1
    return(sult1)

    
def BM25(keyword):
    tokenized_corpus = [list(jieba.cut(doc)) for doc in corpus_data]
    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = list(jieba.cut(keyword))
    doc_scores = bm25.get_scores(tokenized_query)
    top10 = bm25.get_top_n(tokenized_query,corpus_data,n=10)

    # print("斷詞結果:",tokenized_corpus)

    list_doc_scores = list(doc_scores)
    list_corpus = list(corpus_data)

    #df sort
    dict_df2={"Similarity":list_doc_scores,
             "Name":list_corpus
            }
    data2=DataFrame(dict_df2)
    sult2=data2.sort_values(by=['Similarity'],ascending=False)
    sult2 = sult2.reset_index(drop=True) #reset df index  
    return(sult2)


def jaccard(keyword):
    def jaccard_similarity(corpus_data,keyword):
        def add_space(s):
            return ' '.join(list(s))

        # 將字中間加入空格
        corpus_data, keyword = add_space(corpus_data), add_space(keyword)
        # 轉化為TF矩陣
        cv = CountVectorizer(tokenizer=lambda s: s.split())
        corpus = [corpus_data, keyword]
        vectors = cv.fit_transform(corpus).toarray()
        # 求交集
        numerator = np.sum(np.min(vectors, axis=0))
        # 求聯集
        denominator = np.sum(np.max(vectors, axis=0))
        # 計算傑卡德係數
        return 1.0 * numerator / denominator

    list_name=[]
    list_similarity=[]
    i=-1
    for abc in corpus_data:
        i=i+1
        Similarity=jaccard_similarity(corpus_data[i], keyword)
        list_name.append(corpus_data[i])
        list_similarity.append(Similarity)

    #df sort
    dict_df3={"Similarity":list_similarity,
              "Name":list_name
               }
    data3=DataFrame(dict_df3)
    sult3=data3.sort_values(by=['Similarity'],ascending=False)
    sult3 = sult3.reset_index(drop=True) #reset df index
    return(sult3)


def TFIDF(keyword):
    class TF_IDF_Model(object):
        def __init__(self, corpus_data):
            self.corpus_data = corpus_data
            self.documents_number = len(corpus_data)
            self.tf = []
            self.idf = {}
            self.init()

        def init(self):
            df = {}
            for document in self.corpus_data:
                temp = {}
                for word in document:
                    temp[word] = temp.get(word, 0) + 1/len(document)
                self.tf.append(temp)
                for key in temp.keys():
                    df[key] = df.get(key, 0) + 1
            for key, value in df.items():
                self.idf[key] = np.log(self.documents_number / (value + 1))

        def get_score(self, index, keyword):
            score = 0.0
            for q in keyword:
                if q not in self.tf[index]:
                    continue
                score += self.tf[index][q] * self.idf[q]
            return score

        def get_documents_score(self, keyword):
            score_list = []
            for i in range(self.documents_number):
                score_list.append(self.get_score(i, keyword))
            return score_list
        
    tf_idf_model = TF_IDF_Model(corpus_data)
    scores = tf_idf_model.get_documents_score(keyword)

    #df sort
    dict_df4={"Similarity":scores,
             "Name":corpus_data
            }
    data4=DataFrame(dict_df4)
    sult4=data4.sort_values(by=['Similarity'],ascending=False)
    sult4 = sult4.reset_index(drop=True) #reset df index
    return(sult4)


def Synonyms(keyword):
    Similarity_list=[]
    name_list=[]

    i=-1
    for abc in corpus_data:
        i=i+1
        r = synonyms.compare(corpus_data[i], keyword, seg=False)
        Similarity_list.append(r)
        name_list.append(corpus_data[i])

    dict_df5={"Similarity":Similarity_list,
             "Name":name_list
            }
    data5=DataFrame(dict_df5)
    sult5=data5.sort_values(by=['Similarity'],ascending=False)
    sult5 = sult5.reset_index(drop=True) #reset df index
    return(sult5)


def word2vec(keyword):
    seed = 666
    sg = 0
    window_size = 1
    vector_size = 20
    min_count = 1
    workers = 5
    epochs = 200
    batch_words = 10000
    
    # Train
#     train_data = word2vec.LineSentence('7_Algorithm.txt')

    model = fasttext.FastText(
        train_data,
        min_count=min_count,
        vector_size=vector_size,
        workers=workers,
        epochs=epochs,
        window=window_size,
        sg=sg,
        seed=seed,
        batch_words=batch_words,
    )
    model.save('fasttext.model')


    # 加載已訓練好的模型
    model = Word2Vec.load('fasttext.model')

    # 加載已訓練好的模型
    model = Word2Vec.load('fasttext.model')

    # 計算相關詞
    res2 = model.wv.most_similar(keyword, topn=10)

    abc=str(res2).split(',')
    list1=[]
    list2=[]

    i=-1
    for ggg in abc:
        i=i+1
        if i%2 ==0:
            list1.append(abc[i])
        else:
            list2.append(abc[i])

    dict_df6={
             "Similarity":list2,
             "Name":list1

            }
    data6=DataFrame(dict_df6)
    sult6=data6.sort_values(by=['Similarity'],ascending=False)
    sult6['Name'] = sult6['Name'].str.replace(r'[^\w\s]+', '')
    sult6['Similarity'] = sult6['Similarity'].str.replace(')', '')
    sult6 = sult6.reset_index(drop=True) #reset df index
    return(sult6)


def sbert(keywork):
    model_name='paraphrase-multilingual-mpnet-base-v2' #sbert官方預訓練模型 https://www.sbert.net/docs/pretrained_models.html 

    model = SentenceTransformer(model_name) #load pre-model
    sentence_vecs = model.encode(corpus_data) #向量化
    query_vecs = model.encode(keywork)        #向量化

    similarity_list=[]
    Name_list=corpus_data

    i=-1
    for cde in sentence_vecs:
        i=i+1
        result=cosine_similarity( [query_vecs[0]], [sentence_vecs[i]] )
        result=result.tolist()
        similarity_list.append(result)

    dict_df7={
             "Similarity":similarity_list,         
             "Name":Name_list
            }

    data7=DataFrame(dict_df7)
    sult7=data7.sort_values(by=['Similarity'],ascending=False) #sort
    sult7 = sult7.reset_index(drop=True) #reset df index
    return(sult7)

def Algorithm7_Vote_Top1(keyword):
    df_top1 = pd.concat([Levenshtein(keyword)[0:1],jaccard(keyword)[0:1],BM25(keyword)[0:1],TFIDF(keyword)[0:1],Synonyms(keyword)[0:1],word2vec(keyword)[0:1],sbert([keyword])[0:1]],axis=0)
    freq_top1 = df_top1.groupby(['Name']).count() 
    freq_top1 = freq_top1.sort_values(by=['Similarity'],ascending=False) #sort
    return(freq_top1)

def Algorithm7_Vote_Top10(keyword):
    df_top10 = pd.concat([Levenshtein(keyword)[0:10],jaccard(keyword)[0:10],BM25(keyword)[0:10],TFIDF(keyword)[0:10],Synonyms(keyword)[0:10],word2vec(keyword)[0:10],sbert([keyword])[0:10]],axis=0)
    freq_top10 = df_top10.groupby(['Name']).count()
    freq_top10 = freq_top10.sort_values(by=['Similarity'],ascending=False) #sort
    return(freq_top10)

In [13]:
%%time
import time
Algorithm7_Vote_Top1("珍珠堡")

Wall time: 18.2 s


,Similarity
Name,
海洋珍珠堡,4
燒肉珍珠堡牛,1
米斯球,1
薑燒珍珠堡,1


In [14]:
%%time
    
Algorithm7_Vote_Top10("珍珠堡")

Wall time: 17.6 s


,Similarity
Name,
杏鮑菇珍珠堡,12
海洋珍珠堡,12
藜麥海洋珍珠堡,9
薑燒珍珠堡,6
蜜芋紅豆珍珠堡,5
燒肉珍珠堡牛,5
藜麥薑燒珍珠堡,3
MOS洋蔥圈,1
藜麥杏鮑菇珍珠堡,1
